In [15]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [16]:
# heading for organization of 
my_columns = ['Ticker', 'Price Per Share','Market Cap', 'Number Of Shares to Purchase']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [17]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

In [18]:
final_dataframe = pd.DataFrame(columns = my_columns)
print('Start of data entry.')
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
print('End of data entry.')

tart of data entry


KeyboardInterrupt: 

In [6]:
# Split list up into smaller parts that are easier to compute avoiding runtime exception.
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,172.79,51487195999,N/A
1,AAL,21.52,14058063723,N/A
2,AAP,227.99,14269099652,N/A
3,AAPL,149.00,2472369628334,N/A
4,ABBV,112.10,196566659910,N/A
...,...,...,...,...
500,YUM,130.93,38537535105,N/A
501,ZBH,158.22,31988275928,N/A
502,ZBRA,553.34,30117104549,N/A
503,ZION,65.73,10208319934,N/A


In [8]:
# Determines how big the budget is for the portfolio used later to determine how many shares of each stock to buy.
portfolio_size = input("Enter your buying power:")

try:
    val = float(portfolio_size)
except ValueError:
    print("Please enter a number. \n Try again:")
    portfolio_size = input("Enter your buying power:")

Enter the value of your portfolio:1000000


In [9]:
# Use portfolio size to determine how much of each stock can be purchased inside of the budget.
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,172.79,51487195999,11
1,AAL,21.52,14058063723,92
2,AAP,227.99,14269099652,8
3,AAPL,149.00,2472369628334,13
4,ABBV,112.10,196566659910,17
...,...,...,...,...
500,YUM,130.93,38537535105,15
501,ZBH,158.22,31988275928,12
502,ZBRA,553.34,30117104549,3
503,ZION,65.73,10208319934,30


In [10]:
writer = pd.ExcelWriter('Manual_ETF.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='DIY SPY', index = False)

In [11]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [12]:
# writer.sheets['DIY SPY'].write('A1', 'Ticker', string_format)
# writer.sheets['DIY SPY'].write('B1', 'Price', string_format)
# writer.sheets['DIY SPY'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['DIY SPY'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['DIY SPY'].set_column('A:A', 20, string_format)
# writer.sheets['DIY SPY'].set_column('B:B', 20, dollar_format)
# writer.sheets['DIY SPY'].set_column('C:C', 20, dollar_format)
# writer.sheets['DIY SPY'].set_column('D:D', 20, integer_format)


In [13]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [14]:
writer.save()